In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib



In [4]:
exp_dir = "experiments/191023_174532_fold_model_type_film_test_run_all_folds/*"
exp_dir = "experiments/191026_141145_fold_seed_model_type_test_two_seeds/*"
exp_dir = "experiments/191028_204715_gamma_steplr_fold_num_epoch_lr_bs_model_type_seed_test_restartable/*"

In [5]:
class HyperParametersSummary:
    def __init__(self, filter_path: str):
        self.predictions = []
        self.parameters = []
        self.table = []
        for f in tqdm(glob(filter_path)):
            parameters = f.split('/')[-1]
            self.parameters.append(parameters)
            column = {}
            for parameter in parameters.split(','):                
                parameter_key, parameter_value = parameter.split('=')
                column[parameter_key] = parameter_value
            # Insert some code here to parse the metrics and their values
            test_file = os.path.join(f, 'testing', "fo=%s" %column["fold"], "final_test_miou.txt")
            if os.path.isfile(test_file):
                with open(test_file) as f:
                    content = f.readlines()
                    for line in content:
                        key, value = line.split(':')
                        column[key] = float(value)
            else:
                pass
            self.table.append(column)
        self.table = pd.DataFrame(self.table)
        self.group_names = ', '.join(self.table.columns)
        print(f'Parameters: {self.group_names}')


In [14]:
exp_dir = "experiments/191101_114656_fold_seed_lr_split_model_type_sgd_train_validaton/*"

hp_summary = HyperParametersSummary(exp_dir)

# hp_summary.table.groupby(['model_type', 'film', 'fold']).mean().unstack()
# hp_summary.table.groupby(['model_type', 'seed', 'fold']).mean().unstack()
results_aggregated = hp_summary.table.groupby(['model_type', 'split', 'lr', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

100%|██████████| 32/32 [00:00<00:00, 2743.00it/s]

Parameters: model_type, fold, seed, lr, split, IOU


IOU                              mIOU
fold                               0       1       2       3          
model_type     split lr                                               
iter_nwe_coatt train 0.00025  0.4156  0.6349  0.4427  0.3834  0.469150
                     0.001    0.4646  0.6314  0.4718  0.4069  0.493675
               val   0.00025  0.4155  0.6499  0.5064  0.4767  0.512125
                     0.001    0.5083  0.6522  0.4927  0.4696  0.530700
nwe            train 0.00025  0.3068  0.6218  0.4438  0.3847  0.439275
                     0.001    0.4680  0.6315  0.5001  0.4615  0.515275
               val   0.00025  0.4377  0.6490  0.4982  0.4805  0.516350
                     0.001    0.5347  0.6345  0.4991  0.4234  0.522925

In [34]:
results_aggregated = hp_summary.table.groupby(['model_type', 'seed', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

IOU                              mIOU
fold                         0       1       2       3          
model_type     seed                                             
coatt          120937      NaN  0.5885  0.4614     NaN  0.524950
               5982340  0.4540  0.5986  0.4719  0.4613  0.496450
iter_nwe_coatt 120937      NaN     NaN  0.4870     NaN  0.487000
               5982340     NaN     NaN  0.5054     NaN  0.505400
nwe            120937   0.4819  0.6498  0.4882  0.4413  0.515300
               5982340  0.5285  0.6521  0.4742  0.4573  0.528025
nwe_coatt      120937      NaN     NaN  0.4954     NaN  0.495400
               5982340  0.4804  0.6542  0.4969     NaN  0.543833

In [6]:
results_aggregated = hp_summary.table.groupby(['model_type', 'film', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

IOU                              mIOU
fold                      0       1       2       3          
model_type     film                                          
coatt          0     0.4657  0.5941  0.4761  0.4384  0.493575
               1     0.4425  0.5838  0.4780  0.4259  0.482550
iter_nwe_coatt 0     0.5004  0.6537  0.4866  0.4765  0.529300
               1     0.4654  0.6474  0.4991  0.4760  0.521975
nwe            0     0.4766  0.6469  0.4904  0.4723  0.521550
               1     0.5081  0.6472  0.4604  0.4697  0.521350
nwe_coatt      0     0.4679  0.6408  0.4785  0.4662  0.513350
               1     0.4484  0.5929  0.4542  0.3734  0.467225